# Risk Markov Projection â€” Interactive Notebook
Configure, run, and inspect the projection pipeline without changing the library code.

**Usage tips**
- Edit the config overrides in the cells below instead of hard-coding in modules.
- Choose `parquet` or `oracle` as the data source.
- Optional: generate a synthetic parquet sample if you do not have input data handy.
- Outputs (CSV + Parquet) are written to the directory you set in the notebook.

In [ ]:
# Environment setup: locate project root that contains config.py and src/
import sys
from pathlib import Path

def find_project_root(start: Path, marker: str = "config.py", max_depth: int = 7) -> Path:
    """Search current and parent directories (and their risk_markov_projection child) for marker and src/."""
    candidates = []
    current = start
    for _ in range(max_depth):
        candidates.append(current)
        candidates.append(current / "risk_markov_projection")
        current = current.parent
    for cand in candidates:
        if (cand / marker).exists() and (cand / "src").exists():
            return cand
    raise FileNotFoundError(f"Could not find project root from {start}")

CWD = Path.cwd().resolve()
ROOT = find_project_root(CWD)
SRC = ROOT / "src"
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
print("Detected project root:", ROOT)
print("Src path added:", SRC)
print("Working directory:", CWD)

In [ ]:
# Imports (force-load project config to avoid name conflicts)
import importlib
import importlib.util
from importlib.machinery import SourceFileLoader
from pathlib import Path
import pandas as pd
import numpy as np

# Explicitly load config.py from project root and register in sys.modules
config_path = ROOT / "config.py"
spec = importlib.util.spec_from_file_location("config", config_path)
config = importlib.util.module_from_spec(spec)
sys.modules["config"] = config
assert spec.loader is not None
spec.loader.exec_module(config)

from src.data.data_loader import load_raw_data
from src.data.schema import default_schema
from src.data.validators import validate_input
from src.pipelines.run_projection import run
from src.utils.logger import get_logger
from src.utils.export_excel import export_projection_excel
from src.utils.cohort_report import export_cohort_del30_report
from src.utils.cohort_report import export_cohort_del30_excel_combined
from src.utils.cohort_report import export_cohort_del30_excel_split
from src.utils.lifecycle_report import build_lifecycle_for_report
from src.utils.lifecycle_report import export_lifecycle_all_products_one_file_extended

logger = get_logger("notebook")


## Configure runtime parameters
Adjust the variables below to control data source, thresholds, and output paths at runtime.

In [ ]:
# Choose data source and paths
DATA_SOURCE = "oracle"  # options: "parquet" or "oracle"
PARQUET_PATH = Path(r"C:/Users/MAFC4709/Python_work/Projection/data/parquet/POS")
GENERATE_SYNTHETIC = False  # set True to create a sample parquet if you have no data

# Oracle SQL overrides (used only when DATA_SOURCE == "oracle")
import textwrap

config.ORACLE_CONFIG["sql"] = textwrap.dedent("""
    SELECT
      CUTOFF_DATE,
      AGREEMENT_ID,
      DISBURSAL_DATE,
      DISBURSAL_AMOUNT,
      DPD_EOM,
      RISK_BUCKET,
      PRINCIPLE_OUTSTANDING,
      STATUS,
      MOB,
      MAFC_SUB_CATEGORY AS PRODUCT_TYPE,
      NPA_STAGEID,
      DPD_GROUP,
      STATE_MODEL,
      MSCORE_GROUP AS RISK_SCORE,
      SALE_CHANNEL
    FROM RISK.TV_MARKOV_POS_RR a
    WHERE a.CUTOFF_DATE <= DATE '2025-10-01'
      AND a.CUTOFF_DATE >= DATE '2024-01-01'
""")

config.ORACLE_CONFIG["params"] = config.ORACLE_CONFIG.get("params", {})
config.ORACLE_CONFIG["sql_dir"] = config.ORACLE_CONFIG.get("sql_dir", "sql")

# Thresholds and other runtime knobs
config.MIN_OBS = 50
config.MIN_EAD = 100.0
config.MAX_MOB = 24
config.CALIBRATION["enabled"] = True

# Output overrides (keeps notebook outputs separate)
config.OUTPUT["dir"] = ROOT / "outputs" / "notebook"
config.OUTPUT["csv_name"] = "projection_notebook.csv"
config.OUTPUT["parquet_name"] = "projection_notebook.parquet"

# Apply data source choice
config.DATA_SOURCE = DATA_SOURCE
config.PARQUET_PATH = PARQUET_PATH

config.DATA_SOURCE

## Caching checkpoints
Set cache paths and toggles to avoid rerunning heavy steps. Clear cache if you change data source or config.

In [ ]:
# Cache settings
CACHE_DIR = ROOT / 'outputs' / 'cache'
CACHE_DIR.mkdir(parents=True, exist_ok=True)
USE_RAW_CACHE = True
RAW_CACHE_PATH = CACHE_DIR / 'raw_df.parquet'
USE_PROJ_CACHE = True
PROJ_CACHE_PATH = CACHE_DIR / 'projection_df.parquet'


## Optional: generate a synthetic parquet sample
Use this if you want a quick run without connecting to Oracle or preparing data.

In [ ]:
def make_synthetic_dataset(num_loans_per_segment: int = 10, max_mob: int = 6) -> pd.DataFrame:
    records = []
    date_base = pd.Timestamp("2024-01-31")
    segments = [("A", "P1"), ("A", "P2"), ("B", "P1"), ("B", "P2")]
    state_cycle = ["CURRENT", "CURRENT", "DPD30+", "DPD60+", "DPD90+", "WRITEOFF", "WRITEOFF"]

    for risk_score, product in segments:
        for i in range(num_loans_per_segment):
            agreement_id = f"{risk_score}{product}{i}"
            ead0 = 1000 + 50 * i
            for mob in range(max_mob + 1):
                state = state_cycle[min(mob, len(state_cycle) - 1)]
                if mob == max_mob and i % 3 == 1:
                    state = "CLOSED"
                cutoff_date = date_base + pd.DateOffset(months=mob)
                ead_value = max(ead0 - mob * 25, 50)
                records.append(
                    {
                        "AGREEMENT_ID": agreement_id,
                        "MOB": mob,
                        "STATE_MODEL": state,
                        "PRINCIPLE_OUTSTANDING": float(ead_value),
                        "CUTOFF_DATE": cutoff_date,
                        "RISK_SCORE": risk_score,
                        "PRODUCT_TYPE": product,
                    }
                )
    return pd.DataFrame(records)


if DATA_SOURCE == "parquet" and GENERATE_SYNTHETIC:
    PARQUET_PATH.mkdir(parents=True, exist_ok=True)
    sample_df = make_synthetic_dataset(num_loans_per_segment=8, max_mob=8)
    sample_file = PARQUET_PATH / "sample.parquet"
    sample_df.to_parquet(sample_file, index=False)
    print("Synthetic parquet created at", sample_file)
    print(sample_df.head())
else:
    print("Synthetic generation skipped; using existing parquet files at", PARQUET_PATH)

## Load and validate data

In [ ]:
schema = default_schema()

if USE_RAW_CACHE and RAW_CACHE_PATH.exists():
    raw_df = pd.read_parquet(RAW_CACHE_PATH)
    print(f'Loaded raw_df from cache: {RAW_CACHE_PATH} shape={raw_df.shape}')
else:
    raw_df = load_raw_data(schema=schema, source=DATA_SOURCE, parquet_path=PARQUET_PATH)
    validate_input(raw_df, schema=schema, max_mob=config.MAX_MOB)
    if USE_RAW_CACHE:
        RAW_CACHE_PATH.parent.mkdir(parents=True, exist_ok=True)
        raw_df.to_parquet(RAW_CACHE_PATH, index=False)
        print(f'Saved raw_df cache to {RAW_CACHE_PATH} shape={raw_df.shape}')

raw_df.shape


In [ ]:
raw_df.head()

## Run projection pipeline
Use the library runner to build transitions, project EAD, apply calibration, and write outputs.

In [ ]:
if USE_PROJ_CACHE and PROJ_CACHE_PATH.exists():
    projection_df = pd.read_parquet(PROJ_CACHE_PATH)
    print(f'Loaded projection from cache: {PROJ_CACHE_PATH} shape={projection_df.shape}')
else:
    projection_df = run(
        asof_date="2024-12-31",
        target_mob=config.MAX_MOB,
        source=DATA_SOURCE,
        parquet_path=PARQUET_PATH,
    )
    if USE_PROJ_CACHE:
        PROJ_CACHE_PATH.parent.mkdir(parents=True, exist_ok=True)
        projection_df.to_parquet(PROJ_CACHE_PATH, index=False)
        print(f'Saved projection cache to {PROJ_CACHE_PATH} shape={projection_df.shape}')

projection_df.head()


## Inspect outputs
- EAD per state, distribution over EAD0, delinquency indicators
- Audit columns: matrix_source, mob_used, n_obs_used, ead_sum_used
- Calibration factor (if enabled)

In [ ]:
state_cols = [col for col in projection_df.columns if col.startswith("EAD_") and col != "EAD0"]
indicator_cols = [col for col in projection_df.columns if col.startswith("DEL_")]
audit_cols = ["matrix_source", "mob_used", "n_obs_used", "ead_sum_used", "calibration_factor"]
display(projection_df[state_cols + indicator_cols + audit_cols].head())

# Fallback coverage
fallback_rate = (projection_df["matrix_source"] != "segment_mob").mean()
print(f"Fallback usage: {fallback_rate*100:.2f}%")

## Visualize delinquency over MOB

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

segment = projection_df[["RISK_SCORE", "PRODUCT_TYPE"]].drop_duplicates().iloc[0]
mask = (projection_df["RISK_SCORE"] == segment["RISK_SCORE"]) & (projection_df["PRODUCT_TYPE"] == segment["PRODUCT_TYPE"])
subset = projection_df.loc[mask]

plt.figure(figsize=(8, 4))
plt.plot(subset["MOB"], subset["DEL_30P_ON_EAD0"], label="DEL_30P_ON_EAD0")
plt.plot(subset["MOB"], subset["DEL_60P_ON_EAD0"], label="DEL_60P_ON_EAD0")
plt.plot(subset["MOB"], subset["DEL_90P_ON_EAD0"], label="DEL_90P_ON_EAD0")
plt.xlabel("MOB")
plt.ylabel("Ratio over EAD0")
plt.title(f"Delinquency trajectory for segment {segment['RISK_SCORE']} / {segment['PRODUCT_TYPE']}")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Export Excel report
output_dir = Path(config.OUTPUT['dir'])
report_path = output_dir / config.OUTPUT.get('report_name', 'indicator_report.csv')
actual_series = None
if report_path.exists():
    rep_df = pd.read_csv(report_path)
    if 'MOB' in rep_df.columns and 'ACTUAL_DEL30P_ON_EAD0' in rep_df.columns:
        actual_series = rep_df.set_index('MOB')['ACTUAL_DEL30P_ON_EAD0']

excel_path = output_dir / 'projection_report.xlsx'
export_projection_excel(projection_df, output_path=excel_path, actual_del30p_by_mob=actual_series)
print('Excel report saved to', excel_path)


## Cohort DEL30 report (Cohort x MOB)
T?o b?ng Cohort (th?ng gi?i ng?n) x MOB v?i %DEL30/EAD0 cho actual v? forecast.

In [ ]:
schema = default_schema()
cohort_col = getattr(schema, 'cohort_col', None)
output_dir = Path(config.OUTPUT['dir'])
cohort_report_path = output_dir / config.OUTPUT.get('cohort_report_name', 'cohort_del30_report.csv')
cohort_report_excel = output_dir / config.OUTPUT.get('cohort_report_excel_name', 'cohort_del30_report.xlsx')
cohort_report_excel_split = output_dir / config.OUTPUT.get('cohort_report_excel_split_name', 'cohort_del30_report_split.xlsx')

if cohort_col and cohort_col in raw_df.columns:
    export_cohort_del30_report(
        raw_df,
        projection_df,
        output_path=cohort_report_path,
        schema=schema,
        state_order=config.STATE_ORDER,
        buckets_30p=config.BUCKETS_30P,
        max_mob=config.MAX_MOB,
    )
    export_cohort_del30_excel_combined(
        raw_df,
        projection_df,
        output_path=cohort_report_excel,
        schema=schema,
        state_order=config.STATE_ORDER,
        buckets_30p=config.BUCKETS_30P,
        max_mob=config.MAX_MOB,
    )
    export_cohort_del30_excel_split(
        raw_df,
        projection_df,
        output_path=cohort_report_excel_split,
        schema=schema,
        state_order=config.STATE_ORDER,
        buckets_30p=config.BUCKETS_30P,
        max_mob=config.MAX_MOB,
    )
    print('Cohort CSV:', cohort_report_path)
    print('Cohort Excel combined:', cohort_report_excel)
    print('Cohort Excel split:', cohort_report_excel_split)
else:
    print('Cohort column not configured or missing; skip cohort report.')


## Lifecycle Actual + Forecast (demo)
Gh?p actual/forecast v? xu?t lifecycle Excel cho t?t c? product/metric.

In [ ]:
# Demo lifecycle export (requires df_actual and df_plan_fc prepared with DPD buckets)
# Replace the placeholders with your actual/forecast lifecycle DataFrames.
try:
    df_actual = df_actual  # provided externally
    df_plan_fc = df_plan_fc  # provided externally
    buckets = ['DPD30+', 'DPD60+', 'DPD90+']
    lifecycle_df = build_lifecycle_for_report(df_actual, df_plan_fc, buckets)
    actual_info = {}  # e.g., {('PRODUCT', pd.Timestamp('2024-01-01')): 6}
    lc_path = output_dir / 'lifecycle_report.xlsx'
    export_lifecycle_all_products_one_file_extended(lifecycle_df, actual_info, lc_path)
    print('Lifecycle report saved to', lc_path)
except NameError:
    print('Define df_actual and df_plan_fc with lifecycle buckets before running this cell.')
